# NeuroMANCER demonstration

In [ ]:
from IPython.display import Image
import warnings
Image('../../figs/Problem_neuromancer4.PNG', width=500)

In [ ]:
from neuromancer.dataset import read_file, normalize_data, split_sequence_data, SequenceDataset
from neuromancer.estimators import LinearEstimator
from neuromancer.blocks import Linear, ResMLP, MLP
from neuromancer.dynamics import BlockSSM
from neuromancer.problem import Problem
from neuromancer.simulators import OpenLoopSimulator
from neuromancer.loggers import BasicLogger
from neuromancer.visuals import VisualizerOpen
from neuromancer.trainer import Trainer
from neuromancer.activations import SoftExponential
from neuromancer.constraint import Variable, Objective
from neuromancer.callbacks import SysIDCallback
from argparse import Namespace
import slim 
import torch
import psl

from torch.utils.data import DataLoader
warnings.filterwarnings("ignore")
Image('../../figs/class_diagram.png')

# System ID

In [ ]:
Image('../../figs/sysid.png')

## Datasets


In [ ]:
data = read_file(psl.datasets['aero'])
data, _ = normalize_data(data, 'zero-one')
nsteps = 64

split_data = split_sequence_data(data, nsteps)

datasets = [SequenceDataset(d, nsteps=nsteps, name=n) for d, n in zip(split_data, ['train', 'dev', 'test'])]
train_loop, dev_loop, test_loop = [d.get_full_sequence() for d in datasets]

train_data, dev_data, test_data = [DataLoader(d, batch_size=len(d), shuffle=False, collate_fn=d.collate_fn) 
                                  for d in datasets]

ny = train_data.dataset.dims["Yf"][-1]
nu = train_data.dataset.dims["Uf"][-1]
nx = 64

## State Estimator

In [ ]:
Image('../../figs/state_estimator.png', width=200)

In [ ]:
# create an estimator for initial system states
dims = {**train_data.dataset.dims, "x0": (nx,)}
state_estimator = LinearEstimator(dims, # dict mapping dataset variable names to their dimensions
                                  input_keys=["Yp"])  # names of dataset variables used as input)

## State Space Model

Block-structured state space models decouple the dynamics of exogenous inputs, state transitions, and system measurements. 

In [ ]:
Image('../../figs/ssm.png', width=300)

In [ ]:
# define state transition map
fx = MLP(nx, nx)

# define output map
fy = Linear(nx, ny)

# define input map
fu = MLP(nu, nx)

# create block-structured state space model
ssm = BlockSSM(fx, fy, fu=fu, input_key_map={"x0": f"x0_{state_estimator.name}"}, name="dynamics")

## Objective Terms
To optimize the weights of our model, we'll use mean-squared error as our objective to minimize the error between ground-truth and neural SSM predictions.

In [ ]:
predictions = Variable(f"Y_pred_{ssm.name}")
truth = Variable("Yf")
xhat = Variable(f"X_pred_{ssm.name}")
 
smoothing_loss = (xhat[:-1] == xhat[1:])^2
ssm_objective = (predictions == truth)^2
ssm_objective.name = 'ssm_loss'
lower_bound = (predictions > 0.)^2
upper_bound = (predictions < 1.)^2

## Optimization Problem

Now we compose an optimization problem from model components, objective terms, and constraints using NeuroMANCER's `Problem` class.

In [ ]:
Image('../../figs/problem_uml.png')

In [ ]:
components = [state_estimator, ssm]
objectives = [ssm_objective]

constraints = [0.01*upper_bound, 0.01*lower_bound, 0.003*smoothing_loss]

model = Problem(objectives, constraints, components)

## Training

The `Trainer` class encapsulates boilerplate training, evaluation, and logging code, and additionally provides options for customized steps (e.g. visualizations, open loop simulations) via a Callback object. 

In [ ]:
Image('../../figs/trainer_uml.png', width=700)

In [ ]:
optim = torch.optim.AdamW(model.parameters(), lr=0.000001)
simulator = OpenLoopSimulator(model, train_loop, dev_loop, test_loop, eval_sim=False)
logger = BasicLogger(verbosity=50, savedir='test', stdout=["nstep_dev_ssm_loss", 
                                                           "open_dev_ssm_loss"])
visualizer = VisualizerOpen(
    ssm,
    1,
    'test'
)

trainer = Trainer(
    model,
    train_data,
    dev_data,
    test_data,
    optim,
    callback=SysIDCallback(simulator, visualizer),
    logger=logger,
    epochs=100,
    eval_metric="nstep_dev_ssm_loss",
    patience=100,
)                             

In [ ]:
best_model = trainer.train()

## Model Evaluation

In [ ]:
linear_model = best_model

In [ ]:
best_outputs = trainer.test(best_model)


In [ ]:
Image('test/open_loop.png')

## Analysis of learned dynamics

In [ ]:
Image('test/nstep_loop.png')

In [ ]:
Image('test/eigmat.png')


## Inductive priors

In [ ]:
# Pick an unreasonable range for the dominant eigenvalue of the linear map
linargs = {'sigma_min': 1.5, 'sigma_max': 2}

fx = Linear(nx, nx, linear_map=slim.maps['pf'], linargs=linargs)

In [ ]:
state_estimator = LinearEstimator(dims, # dict mapping dataset variable names to their dimensions
                                  input_keys=["Yp"])  # names of dataset variables used as input)
# define output map
fy = Linear(nx, ny)

# define input map
fu = Linear(nu, nx)

# create block-structured state space model
ssm = BlockSSM(fx, fy, fu=fu, input_key_map={"x0": f"x0_{state_estimator.name}"}, name="dynamics")

model = Problem(objectives, constraints, components)
components = [state_estimator, ssm]
objectives = [ssm_objective]
constraints = []

In [ ]:
optim = torch.optim.AdamW(model.parameters(), lr=0.001)
simulator = OpenLoopSimulator(model, train_loop, dev_loop, test_loop, eval_sim=False)
logger = BasicLogger(verbosity=50, savedir='test', stdout=["nstep_dev_ssm_loss", 
                                                           "open_dev_ssm_loss"])
visualizer = VisualizerOpen(
    ssm,
    1,
    'test'
)

trainer = Trainer(
    model,
    train_data,
    dev_data,
    test_data,
    optim,
    callback=SysIDCallback(simulator, visualizer),
    logger=logger,
    epochs=100,
    eval_metric="nstep_dev_ssm_loss",
    patience=100,
)    

best_model = trainer.train()

In [ ]:
Image('test/eigmat.png')


## Domain Aware Priors

In [ ]:
# Pick a more reasonable range for constraining the singular values of the linear maps
linargs = {'sigma_min': 0.1, 'sigma_max': 1.0}
fx = MLP(nx, nx, hsizes=[64, 64,64],linear_map=slim.maps['softSVD'], linargs=linargs)


reg = Variable(f'reg_error_{ssm.name}')
svd_error = (reg == 0.0)^2

Image('../../figs/svd.png', width=300)

## Constraint Terms
In addition to minimizing the mean-squared error of predicted and expected observables, we may also want to impose further constraints on different model components to enforce certain model behaviors. 

In [ ]:
state_predictions = Variable(f"X_pred_{ssm.name}")
initial_states = Variable(f"x0_{state_estimator.name}")

dx_penalty = (state_predictions[1:] == state_predictions[:-1])^2

In [ ]:
state_estimator = LinearEstimator(dims, # dict mapping dataset variable names to their dimensions
                                  input_keys=["Yp"])  # names of dataset variables used as input)
# define output map
fy = Linear(nx, ny)

# define input map
fu = Linear(nu, nx)

# create block-structured state space model
ssm = BlockSSM(fx, fy, fu=fu, input_key_map={"x0": f"x0_{state_estimator.name}"}, name="dynamics")

model = Problem(objectives, constraints, components)
components = [state_estimator, ssm]
objectives = [ssm_objective]
constraints = [0.1*dx_penalty, 0.1*svd_error]

In [ ]:
optim = torch.optim.AdamW(model.parameters(), lr=0.00001)
simulator = OpenLoopSimulator(model, train_loop, dev_loop, test_loop, eval_sim=False)
logger = BasicLogger(verbosity=50, savedir='test', stdout=["nstep_dev_ssm_loss", 
                                                           "open_dev_ssm_loss"])
visualizer = VisualizerOpen(
    ssm,
    1,
    'test'
)

trainer = Trainer(
    model,
    train_data,
    dev_data,
    test_data,
    optim,
    callback=SysIDCallback(simulator, visualizer),
    logger=logger,
    epochs=100,
    eval_metric="nstep_dev_ssm_loss",
    patience=100,
)    

best_model = trainer.train()
best_outputs = trainer.test(best_model)

Image('test/open_loop.png')

In [ ]:
Image('test/eigmat.png')